In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import sys; sys.path.insert(0,'..')
from main import *

In [2]:
X_train, y_train = load_svmlight_file('../data/w8a/w8a.txt')
X_train = X_train.toarray()

X_test, y_test = load_svmlight_file('../data/w8a/w8a.t')
X_test = X_test.toarray()

# New

In [3]:
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

X_train, y_train, X_test, y_test = X_train[:50], y_train[:50], X_test[:500], y_test[:500]

rank = generate_preliminary_pairs(X_train, y_train, kernel='linear', random_seed=1)
W = generate_from_preliminary_W(X_train, y_train, rank, m=200, kernel='linear', random_seed=1)

PRELIMINARY
Training accuracy of supervised SVM: 1.00

GENERATE W
[[-1. -1.]
 [-1.  1.]
 [ 1. -1.]] [184   8   8]
Training accuracy of pairwise rank SVM: 0.99



In [4]:
# def new_svm_lambdaboost(X_train, y_train, X_test, y_test, W, prior, T=20, sample_prop=1, random_seed=None, verbose=True):
# """ Generates linear svm lambdaboost classifier """
prior = 0.04
T=20; sample_prop=1; random_seed=None; verbose=True


print("NEIL'S NEW MODEL")

if verbose:
    print('LINEAR SVM SUBMODELS')
    print('Using %d classifiers and sample proportion of %d'
          % (T, sample_prop))
    if random_seed:
        print('Random seed %d', random_seed)

# Constants
m = X_train.shape[0]
n = X_train.shape[1]
np.random.seed(random_seed)

# Initialize model parameters
f_intercept = 0
f_coefficient = np.zeros(n)

# Initialize counters
t = 1
alpha = [0.0]
acc_train_ls = []
acc_test_ls  = []

# Training
while t <= T and alpha[-1] >= 0:

    # Step 6: compute epsilon
    curr_pred = np.dot(X_train, f_coefficient) + f_intercept
    # Scale predictions, works well empirically
    curr_pred = minmax_scale(curr_pred, feature_range=(-1, 1))
    # Remember that W is passed as a list of arrays!
    epsilon = cal_uncertainty(curr_pred, [W])

    # Step 7: compute weights
    weight = cal_weights(epsilon)

    # Step 8: extract labels
    #y = np.sign(weight)
#     if t == 1:
#         weight = np.sort(weight)
#         y = np.concatenate((-np.ones(m - int(m * prior)), np.ones(int(m * prior))))
#     else:
#         y = np.sign(weight)
    weight_idx = np.argsort(weight)
    X_new = X_train[weight_idx]
    y_new = np.concatenate((-np.ones(m - int(m * prior)), np.ones(int(m * prior))))

    # Step 9: create training (sample) data by sampling based on weights
    weight = np.sort(weight) # Added, necessary for sampling probability
    p_weight = np.abs(weight)
    p_weight /= np.sum(p_weight)
    sample = np.random.choice(m, size=m*sample_prop, replace=True, p=p_weight)
    X_sample = X_new[sample]
    y_sample = y_new[sample]
    
#     """ REMOVE """
#     X_sample = X_new
#     y_sample = y_new

    # Step 10: learn binary classifier on training (sample) data
    clf = LinearSVC(max_iter=1000)
    clf.fit(X_sample, y_sample)

    # Step 11: predict labels using current classifier
    y_pred = clf.predict(X_train)

    # Step 12: compute weight of current classifier
    alpha_t = cal_alpha(y_pred, epsilon)

    # Make sure alpha is valid
    if np.isnan(alpha_t) or np.isinf(alpha_t):
        print('Alpha invalid, terminated')
        break

    # Step 13: update final classifier
    f_coefficient += alpha_t * clf.coef_.ravel()
    f_intercept += alpha_t * clf.intercept_

    # Update loop
    alpha.append(alpha_t)
    t += 1

    # Evaluation
    y_train_pred = np.dot(X_train, f_coefficient) + f_intercept
    y_test_pred = np.dot(X_test, f_coefficient) + f_intercept
    y_train_pred = np.sign(y_train_pred)
    y_test_pred = np.sign(y_test_pred)

    acc_train_curr = accuracy_score(y_train, y_train_pred)
    acc_test_curr  = accuracy_score(y_test, y_test_pred)
    acc_train_ls.append(max(acc_train_curr, 1 - acc_train_curr))
    acc_test_ls.append(max(acc_test_curr, 1 - acc_test_curr))

    if verbose:
        if t == 2:
            print('t\tTrain\t\tTest\t\tPrior')
        print('%d\t%.2f\t\t%.2f\t\t%.2f' % (t - 1, acc_train_curr, acc_test_curr, (np.sum(y_sample == 1) / y_sample.size)))
        if alpha_t < 0:
            print('Alpha %.2f, terminated' % alpha_t)
            
# To skip initialized 0 in alpha list
alpha = alpha[1:]

# Get final accuracy on best boosting iteration on train set
# Do not record best iteration on test set -- would train hyperparameter on test
max_idx = np.argmax(acc_train_ls)
acc_train_final = acc_train_ls[max_idx]
acc_test_final  = acc_test_ls[max_idx]

if verbose:
    print('t = %d was best iteration with accuracy %.2f\n' % (max_idx + 1, acc_test_final))

# Return minimum error (1 - max accuracy)
1 - acc_train_final, 1 - acc_test_final

NEIL'S NEW MODEL
LINEAR SVM SUBMODELS
Using 20 classifiers and sample proportion of 1
t	Train		Test		Prior
1	0.98		0.98		0.08
2	0.98		0.98		0.12
3	0.98		0.98		0.08
4	0.96		0.97		0.02
5	0.96		0.97		0.12
6	0.96		0.97		0.06
7	0.96		0.97		0.20
8	0.96		0.97		0.12
9	0.96		0.97		0.10
10	0.96		0.97		0.04
11	0.96		0.97		0.04
12	0.96		0.97		0.12
13	0.96		0.97		0.06
14	0.96		0.97		0.14
15	0.96		0.97		0.04
16	0.96		0.97		0.06
17	0.96		0.97		0.08
18	0.96		0.97		0.06
19	0.96		0.97		0.06
20	0.96		0.97		0.02
t = 1 was best iteration with accuracy 0.98



(0.020000000000000018, 0.02400000000000002)

# Old

In [ ]:
eq1 = np.sum(y_test == 1) / y_test.size
eq1

In [ ]:
def svm_lambdaboost(X_train, y_train, X_test, y_test, W, T=20, sample_prop=1, random_seed=None, verbose=True):
    """ Generates linear svm lambdaboost classifier """

    if verbose:
        print('LINEAR SVM SUBMODELS')
        print('Using %d classifiers and sample proportion of %d'
              % (T, sample_prop))
        if random_seed:
            print('Random seed %d', random_seed)

    # Constants
    m = X_train.shape[0]
    n = X_train.shape[1]
    np.random.seed(random_seed)

    # Initialize model parameters
    f_intercept = 0
    f_coefficient = np.zeros(n)

    # Initialize counters
    t = 1
    alpha = [0.0]
    acc_train_ls = []
    acc_test_ls  = []

    # Training
    while t <= T and alpha[-1] >= 0:

        # Step 6: compute epsilon
        curr_pred = np.dot(X_train, f_coefficient) + f_intercept
        # Scale predictions, works well empirically
        curr_pred = minmax_scale(curr_pred, feature_range=(-1, 1))
        # Remember that W is passed as a list of arrays!
        epsilon = cal_uncertainty(curr_pred, [W])

        # Step 7: compute weights
        weight = cal_weights(epsilon)

        # Step 8: extract labels
        y = np.sign(weight)

        # Step 9: create training (sample) data by sampling based on weights
        p_weight = np.abs(weight)
        p_weight /= np.sum(p_weight)
        sample = np.random.choice(m, size=m*sample_prop, replace=True, p=p_weight)
        X_sample = X_train[sample]
        y_sample = y[sample]

        # Step 10: learn binary classifier on training (sample) data
        clf = LinearSVC(max_iter=1000)
        clf.fit(X_sample, y_sample)

        # Step 11: predict labels using current classifier
        y_pred = clf.predict(X_train)

        # Step 12: compute weight of current classifier
        alpha_t = cal_alpha(y_pred, epsilon)

        # Make sure alpha is valid
        if np.isnan(alpha_t) or np.isinf(alpha_t):
            print('Alpha invalid, terminated')
            break

        # Step 13: update final classifier
        f_coefficient += alpha_t * clf.coef_.ravel()
        f_intercept += alpha_t * clf.intercept_

        # Update loop
        alpha.append(alpha_t)
        t += 1

        # Evaluation
        y_train_pred = np.dot(X_train, f_coefficient) + f_intercept
        y_test_pred = np.dot(X_test, f_coefficient) + f_intercept
        y_train_pred = np.sign(y_train_pred)
        y_test_pred = np.sign(y_test_pred)

        acc_train_curr = accuracy_score(y_train, y_train_pred)
        acc_test_curr  = accuracy_score(y_test, y_test_pred)
        acc_train_ls.append(max(acc_train_curr, 1 - acc_train_curr))
        acc_test_ls.append(max(acc_test_curr, 1 - acc_test_curr))

        if verbose:
            if t == 2:
                print('t\tTrain\t\tTest\t\tPrior')
            print('%d\t%.2f\t\t%.2f\t\t%.2f' % (t - 1, acc_train_curr, acc_test_curr, (np.sum(y == 1) / y.size)))
            if alpha_t < 0:
                print('Alpha %.2f, terminated' % alpha_t)

    # To skip initialized 0 in alpha list
    alpha = alpha[1:]

    """ CHANGED """
    # # Return final classifier parameters, weight of each submodel
    # return f_coefficient, f_intercept, alpha

    """ CHANGED (again) """
    # acc_train = accuracy_score(y_train, y_train_pred)
    # acc_test  =  accuracy_score(y_test, y_test_pred)
    # return min(acc_train, 1 - acc_train), min(acc_test, 1 - acc_test)

    # Get final accuracy on best boosting iteration on train set
    # Do not record best iteration on test set -- would train hyperparameter on test
    max_idx = np.argmax(acc_train_ls)
    acc_train_final = acc_train_ls[max_idx]
    acc_test_final  = acc_test_ls[max_idx]

    if verbose:
        print('t = %d was best iteration with accuracy %.2f\n' % (max_idx + 1, acc_test_final))

    # Return minimum error (1 - max accuracy)
    return 1 - acc_train_final, 1 - acc_test_final

In [ ]:
svm_lambdaboost(
    X_train,
    y_train,
    X_test,
    y_test,
    W)